In [1]:
# Importing useful libraries
import pandas as pd
import xlrd
import sys
sys.path.append('..')
import warnings
warnings.filterwarnings('ignore')
import openpyxl # For excel related tasks
from scripts import utils
import matplotlib.pyplot as plt
import seaborn as sns

# Enabling to show all the columns and rows
pd.pandas.set_option(
    'display.max_columns',
    None
)

In [2]:
# Getting the month dictionary

months_dict = {
    'Jan': 'January',
    'Feb': 'February',
    'Mar': 'March',
    'Apr': 'April',
    'May': 'May',
    'Jun': 'June',
    'Jul': 'July',
    'Aug': 'August',
    'Sep': 'September',
    'Oct': 'October',
    'Nov': 'November',
    'Dec': 'December'
}

months_dict_2 = {
    "01": "January",
    "02": "February",
    "03": "March",
    "04": "April",
    "05": "May",
    "06": "June",
    "07": "July",
    "08": "August",
    "09": "September",
    "10": "October",
    "11": "November",
    "12": "December"
}



In [3]:
# Reading the comprehensive report data to get the month

xls = pd.read_excel('../data/february_revenue_data.xlsx',sheet_name=None)

feb_data_frames = {}

for sheet_name,sheet_df in xls.items():
    feb_data_frames[sheet_name] = sheet_df


xls = pd.read_excel('../data/january_revenue_data.xlsx',sheet_name=None)

jan_data_frames = {}

for sheet_name,sheet_df in xls.items():
    jan_data_frames[sheet_name] = sheet_df

In [4]:
# Getting the month name from the comprehensive report data

month_names = [date.split(' ')[0] for date in jan_data_frames.keys()]

print(months_dict[list(set(month_names))[0]])

January


In [5]:
# # Getting the month name from the clik vs reg data
# click_df = utils.read_excel_file('../data/ClicksVsRegistration.xls')
# useful_columns = [
#     'Date', 'Affiliate Name', 'Affiliate ID', 'Revenue Tracker ID',
#     'User Registration', 'Cake Clicks (All Clicks)'
# ]
# click_df = click_df[useful_columns]
# click_df.dropna(inplace=True)
# months = [date.split('-')[1] for date in click_df['Date'].values]
# print(months_dict_2[list(set(months))[0]])

In [6]:
# Getting the affiliate names from the comprehensive reports

feb_campaigns = []

cut_off_1 = feb_data_frames['Feb 1'][feb_data_frames['Feb 1']['Campaign'] == 'Publisher Name'].index[0]
temp_df = feb_data_frames['Feb 1'].iloc[cut_off_1:]
cut_off_2 = temp_df[temp_df['Campaign'].isna() == True].index[0]
campaigns = list(set(list(feb_data_frames['Feb 1'].iloc[cut_off_1:cut_off_2]['Campaign'].values)))
campaigns.remove('Publisher Name')

feb_campaigns += campaigns

jan_campaigns = []

cut_off_1 = jan_data_frames['Jan 01'][jan_data_frames['Jan 01']['Campaign'] == 'Publisher Name'].index[0]
temp_df = jan_data_frames['Jan 01'].iloc[cut_off_1:]
cut_off_2 = temp_df[temp_df['Campaign'].isna() == True].index[0]
campaigns = list(set(list(jan_data_frames['Jan 01'].iloc[cut_off_1:cut_off_2]['Campaign'].values)))
campaigns.remove('Publisher Name')

jan_campaigns += campaigns

In [7]:
jan_final_dfs = utils.generate_report('../data/january_revenue_data.xlsx')
feb_final_dfs = utils.generate_report('../data/february_revenue_data.xlsx')

# Getting the monthly data for February
feb_dfs = {}
for sheet_name,sheet_df in feb_final_dfs.items():
    feb_dfs[sheet_name] = sheet_df

# RGetting the monthly data for January
jan_dfs = {}
for sheet_name,sheet_df in jan_final_dfs.items():
    jan_dfs[sheet_name] = sheet_df

# Let us check only the all in box data
needed_affiliate = 'All Inbox(Jason Jacobs)'
feb_dfs[needed_affiliate]

# Let us get the comparision dfs
final_df = pd.concat([jan_dfs[needed_affiliate],feb_dfs[needed_affiliate]],axis=0)
final_df.drop(
    columns = 'Date',
    inplace = True
)
final_df.dropna(inplace = True)
final_df.sort_values('Publisher',inplace=True)
final_df.to_excel('report.xlsx',index=False)

excel_path = 'report.xlsx'
# Load the Excel file with openpyxl
workbook = openpyxl.load_workbook(excel_path)

# Apply formatting to all sheets
for sheet_name in workbook.sheetnames:
    sheet = workbook[sheet_name]
    utils.format_sheet(sheet)

workbook.save(excel_path)
